# Planification d'une tournée musicale

Dans ce sujet de SAE, sur 2 séances encadrées, nous allons utiliser la formalisation d'un problème de résolution par l'exploration ainsi que les différents algorithmes de recherche vus en cours puis en TD pour traiter un problème d'organisation d'une tournée française d'un groupe de musique sous la forme d'un problème de voyageur de commerce.

Dans le début de ce notebook, l'implémentation de la formalisation générique d'un problème de résolution par l'exploration ainsi que les différents algorithmes de recherche vus en cours puis en TD vous sont fournis. Vous aurez ensuite à compléter la classe `PlanificationTournee` pour résoudre ce problème sur des données qui vous sont fournies. Vous pourrez ensuite ajouter de nouvelles villes, en faisant appel à l'API Python d'Open Street Map, pour récupérer les distances avec vos nouvelles villes.

In [88]:
#from numpy.random import randint
#import matplotlib.pyplot as plt
#import random
import heapq
#import math
#import sys
from collections import defaultdict, deque, Counter
#from itertools import combinations

# Définition d'un problème

On commence par définir une classe abstraite pour représenter un `Problem` qui sera résolu par l'exploration.

On indique l'état initial et soit l'état final (s'il y en a un seul), soit la manière de savoir si on se trouve dans un état final (en redéfinissant la méthode `is_goal`, dans la sous-classe qui implémente la classe `Problem`). La sous-classe devra également implémenter le constructeur `__init__`(en ajoutant éventuellement d'autres paramètres) et les méthodes `actions`, `result` `action_cost` et `h`. 


In [89]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other arguments for the subclass."""

    def __init__(self, initial=None, goal=None): 
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal
        
        
    def actions(self, state):     
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""   
        raise NotImplementedError
    
    def result(self, state, action): 
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError
    
    
    def is_goal(self, state):   
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor."""     
        return state == self.goal
    
    
    def action_cost(self, s, a, s1): 
        """Return the cost of an action a from state s to state s1."""
        return 1
    
    
    def h(self, node):  
        """Returns least possible cost to reach a goal for the state in given node."""             
        return 0
    
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

# Définition d'un noeud

Dans la classe `Node`, on définit un noeud dans un arbre de recherche.
Le noeud contient un pointeur vers son noeud parent `parent` ainsi que vers l'état `state` auquel il correspond (la notion d'état sera défini au moment de la définition du problème représenté). Il est à noter que si un état peut être atteint par 2 chemins, il y aura alors 2 noeuds contenant le même état. Le noeud contient également l'action `action` qui a permis d'atteindre l'état associé au noeud ainsi que le coût total `path_cost` du chemin allant du noeud initial au noeud courant (correspondant à la valeur g dans les algorithmes d'exploration).

In [90]:
class Node:
    "A Node in a search tree."
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost


    def __repr__(self): 
        return '<{}>'.format(self.state)
    
    
    def __len__(self): 
        return 0 if self.parent is None else (1 + len(self.parent))
    
    
    def __lt__(self, other): 
        return self.path_cost < other.path_cost
    

## Fonctions relatives aux noeuds d'un arbre de recherche

On définit également quelques fonctions sur les noeuds : `expand` pour générer les noeuds successeurs d'un noeud donné, `path_actions` pour récupérer la liste d'actions ayant permis d'arriver jusqu'au noeud et `path_states` pour récupérer la liste des états ayant permis d'arriver jusqu'au noeud.


In [91]:
def expand(problem, node):
    "Expand a node, generating the children nodes (see slide 18 in the lecture)."
    s = node.state
    succs = []
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        succs.append(Node(s1, node, action, cost))
    return succs
        

def path_actions(node):
    "The list of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The list of states to get to this node."
    #if node in (failure, None): 
    if node.parent is None:
        return [node.state]
    return path_states(node.parent) + [node.state]

# Différents types de files pour implémenter la frontière

Les files FIFO et LIFO sont utilisées, respectivement, pour implémenter la `frontière` dans l'exploration en largeur et dans l'exploration en profondeur. La file de priorié `PriorityQueue` est utilisée dans l'exploration gloutonne et dans l'exploration A* pour garder la frontière triée en fonction du score croissant `f(item)`.

In [92]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queue."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): 
        return self.items[0][1]

    def __len__(self): 
        return len(self.items)


# Algorithmes de recherche

Les algorithmes de recherche non-informée (exploration en largeur et exploration en profondeur, en mémorisant ou non les états déjà exploras) ainsi que les algorithmes de recherche informée (exploration gloutonne et A*) seront implémentés ici.

In [93]:
# Noeud particulier pour indiquer qu'un algorithme ne peut pas trouver de solution    
failure = Node('failure', path_cost=math.inf)

## Exploration en largeur de l'arbre de recherche


In [94]:
def breadth_first_tree_search(problem):
    """Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops."""
    frontier = FIFOQueue([Node(problem.initial)])  # FIFO queue

    while frontier:
        node = frontier.popleft()
        #print(node)
        if problem.is_goal(node.state):
            return node
        frontier.extend(expand(problem, node))
    return None

## Exploration en profondeur de l'arbre de recherche


In [95]:
def depth_first_tree_search(problem):
    """Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """
    frontier = LIFOQueue([Node(problem.initial)])  # Stack

    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        frontier.extend(expand(problem, node))
    return None

## Exploration en largeur du graphe des états


In [96]:
def breadth_first_graph_search(problem):
    """Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    frontier = FIFOQueue([node])
    explored = set()
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in expand(problem, node):
            if child.state not in explored and child not in frontier:
                if problem.is_goal(node.state):
                    return node
                frontier.append(child)
    return None

## Exploration en profondeur du graphe des états

In [97]:
def depth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = LIFOQueue([(Node(problem.initial))])  # Stack

    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in expand(problem, node)
                        if child.state not in explored and child not in frontier)
    return None

## Exploration gloutonne du graphe des états


In [98]:
def greedy_graph_search(problem, h=None):
    """Search nodes with minimum h(n)."""
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=h)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

## Exploration A* du graphe des états


In [99]:
def astar_graph_search(problem, f=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

# Définition d'une carte

On commence par définir la classe `Map` (pour représenter les villes d'une carte). 
Une carte contient les coordonnées gps des villes considérées (correspondant à l'attribut `coords`) ainsi que les distances réelles (c'est-à-dire en ne passant que par des routes existantes) entre certains couples de villes (correspondant à l'attribut `distances`, représenté par un dictionnaire dont la clé est une paire de noms de villes ; chaque ville est désignée par son nom). Le graphe considéré n'est donc pas toujours un graphe complet (on ne donne pas les distances réelles entre tous les couples de villes qu'on pourrait former). 


#### Question 1 : compléter la méthode `neighbors(city)` qui donne la liste des villes voisines de la ville `city` donnée en paramètre.

In [100]:
class Map:
    
    def __init__(self, distances=None, locations=None):
        self.distances = distances
        self.locations = locations


    def neighbors(self, city):
        #TODO
        raise NotImplementedError

## Test de la classe Map

In [101]:
# Une première carte avec 10 villes

tour1 = Map(
    {('Caen', 'Rennes'): 185, ('Caen', 'Nantes'): 271, ('Caen', 'Le Mans'): 160,
     ('Rennes', 'Caen'): 184, ('Rennes', 'Le Mans'): 157, ('Rennes', 'Nantes'): 123,
     ('Le Mans', 'Caen'): 165, ('Le Mans', 'Rennes'): 162, ('Le Mans', 'Angers'): 90, ('Le Mans', 'Orléans'): 137,
     ('Orléans', 'Le Mans'): 138, ('Orléans', 'Angers'): 221, ('Orléans', 'Clermont-Ferrand'): 301,
     ('Angers', 'Le Mans'): 98, ('Angers', 'Nantes'): 92, ('Angers', 'Orléans'): 221, ('Angers', 'Clermont-Ferrand'): 405,
     ('Nantes', 'Le Mans'): 177, ('Nantes', 'Angers'): 93, ('Nantes', 'Caen'): 274, ('Nantes', 'Rennes'): 122, ('Nantes', 'Clermont-Ferrand'): 468,
     ('La Rochelle', 'Nantes'): 148, ('La Rochelle', 'Bordeaux'): 199, ('La Rochelle', 'Clermont-Ferrand'): 391,
     ('Bordeaux', 'La Rochelle'): 202, ('Bordeaux', 'Toulouse'): 249, ('Bordeaux', 'Clermont-Ferrand'): 371,
     ('Toulouse', 'Bordeaux'): 246, ('Toulouse', 'Clermont-Ferrand'): 368,
     ('Clermont-Ferrand', 'Toulouse'): 367, ('Clermont-Ferrand', 'Bordeaux'): 372, ('Clermont-Ferrand', 'La Rochelle'): 393,
     ('Clermont-Ferrand', 'Nantes'): 450, ('Clermont-Ferrand', 'Angers'): 400, ('Clermont-Ferrand', 'Orléans'): 301
     },
    {'Nantes': ( 47.21, -1.56), 'Angers': (47.49, -0.54), 
     'Orléans': (47.90, 1.89), 'Rennes': (48.10, -1.71), 
     'La Rochelle': (46.16, -1.22), 'Caen': (49.21, -0.34), 
     'Le Mans': (47.96, 0.20), 'Bordeaux': (44.86, -0.52), 
     'Toulouse': (43.57, 1.32), 'Clermont-Ferrand': (45.79, 3.10)
     })


# Planification d'une tournée musicale

Pour résoudre le problème de planification d'une tournée musicale, on définit la classe `MusicTourProblem`. La résolution de ce problème consiste à partir d'une ville de départ et à parcourir toutes les autres villes données, une et une seule fois, puis revenir dans la ville de départ, en minimisant le trajet total.

Un état contient la liste des noms complets des villes déjà parcourues (par exemple, `'Nantes'`). Les actions sont également les noms des villes ; par exemple, l'action `'Nantes'` correspond à l'action de se déplacer dans la ville de `'Nantes'`. Les liaisons possibles entre les villes sont données grâce à l'objet de type `Map`, qui correspond à un graphe orienté, dans lequel les sommets sont les villes (avec leurs coordonnées gps) et les arcs sont les routes existantes entre 2 villes (avec la distance entre les villes).
L'heuristique utilisée correspond à la distance euclidienne minimale entre la dernière ville parcourue et les villes non encore parcourues. 

#### Question 2 : compléter la classe `MusicTourProblem`.

In [102]:
class MusicTourProblem(Problem):
    
    def __init__(self, initial=None, goal=None, map=None):
        super().__init__(initial, goal)
        self.map = map
    
    def actions(self, state): 
        #TODO
        raise NotImplementedError
    
    def result(self, state, action):
        raise NotImplementedError
    
    def is_goal(self, state):   
        raise NotImplementedError
    
    def action_cost(self, s, action, s1):
        raise NotImplementedError
    
    def h(self, node):
        raise NotImplementedError
    

#### Question 3 : tester les différents algorithmes de recherche non informés et informés, en affichant le chemin obtenu et la distance totale sur ce chemin.

In [103]:
# test des algorithmes de recherche sur la carte tour1
music1 = MusicTourProblem('Nantes', 'Nantes', map=tour1)




#### Question 4 : afficher le chemin obtenu, en affichant une carte de la France, les villes choisies et le chemin obtenu (pour chacun des algorithmes de recherche).

In [104]:
# code à compléter

#### Question 5 : ajouter des villes dans la carte de départ et exécuter à nouveau les différents algorithmes de recherche, en affichant les résultats obtenus sur la carte de la France. Pour calculer les coordonnées gps des villes ajoutées ainsi que les distances entre ces villes et les villes déjà présentes, vous pouvez utiliser l'API d'Open Street Map (`OSMPythonTools` : https://wiki.openstreetmap.org/wiki/OSMPythonTools).

In [105]:
# test des algorithmes de recherche sur la carte tour2 (à compléter)
tour2 = Map(
    {},
    {})

music2 = MusicTourProblem('Nantes', 'Nantes', map=tour2)


#### Question 6 : comparer les différents algorithmes de recherche, en termes de nombre de noeuds générés, nombre de noeuds développés et tamps de calcul (pour la carte donnée, tour1, et pour la carte que vous aurez créée, tour2)

In [106]:
# code à compléter